## Classification
- **Culturally Exclusive**: 'caponata' -> Italy
- **Cultural Representative**: 'pizza' -> Italy
- **Cultural Agnostic**: 'bread'

#### Exploiting the graph knowledge-based wikidata structure
- **Entity**: the item (e.g., "pizza")
- **P495**: country of origin
- **P2596**: culture
- **P172**: ethnic group
- **P37**: official language
- **P407**: langauge of work or name
- **P135**: Movement(Art, Literature, philosophy)
- **P136**: Genre
- **P921**: Main subject
- **P547**: Memorialized by
- **P793**: Significant event
- **P840**: Narrative location
- **P17**: Country
- **P1843**: Taxon common name
- **P1001**: Applies to jurisdiction
- **P144**: Based on
- **P361**: Part of
- **P1705**: Native label
- **P2012**: cuisine
- **P2541**: Operating area
- **P1535**: Used by
- **P366**: Use
- **P1142**: Political ideology
- **P140**: Religion
- **P102**: Member of political party
- **P1344**: Participant in
- **P183**: Endemic to
- **P2341**: Indigenous to
- **P1532**: Country for sport
- **P279**: Subclass of

In [ ]:
import requests
import json
from wikidata.client import Client
import pandas as pd

In [ ]:
url = 'https://query.wikidata.org/sparql'

Example of a query for extracting country of origin and culture for `Caponata(Q1034806)`

In [ ]:
query = ''' 
SELECT ?origin ?culture
WHERE {
    wd:Q1034806 wdt:P495 ?origin;
                wdt:P2596 ?culture.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
'''

In [ ]:
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

In [ ]:
properties = ['origin', 'culture']
query_result = {}
for prop in properties:
    query_result[prop] = data['results']['bindings'][0][prop]['value']

print(query_result)

# To transform the result in a json file

'''
    json_format = json.dumps(query_result)
    print(json_format)
'''

# Extracting the IDs

query_result2 = {}
for prop in properties:
    query_result2[prop] = data['results']['bindings'][0][prop]['value'].strip().split("/")[-1]

print(query_result2)

In [ ]:
def print_query_result(data, properties):
    query_result = {}
    keys = data['results']['bindings'][0].keys()
    for prop in properties:
        if prop in keys:
            query_result[prop] = data['results']['bindings'][0][prop]['value']
        else:
            query_result[prop] = None

    return query_result

In [ ]:
client = Client()

labels = {}
for prop in properties:
    entity_ID = query_result2[prop]
    item = client.get(entity_ID, load=True)
    labels[prop] = [entity_ID, str(item.label)]

print(labels)


To deal with missing values we must add the `OPTIONAL` keyword

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
sparql = SPARQLWrapper('https://query.wikidata.org/sparql')

In [ ]:
sparql.setReturnFormat(JSON)

In [ ]:
sparql.setQuery(
'''
SELECT ?origin ?culture ?ethnic_group ?off_language ?related_trad ?name ?country_use ?cuisine
WHERE {
    OPTIONAL { wd:Q177 wdt:P495 ?origin. }
    OPTIONAL { wd:Q177 wdt:P2596 ?culture. }
    OPTIONAL { wd:Q177 wdt:172 ?ethnic_group. }
    OPTIONAL { wd:Q177 wdt:37 ?off_language. }
    OPTIONAL { wd:Q177 wdt:4950 ?related_trad. }
    OPTIONAL { wd:Q177 wdt:407 ?name. }
    OPTIONAL { wd:Q177 wdt:17 ?country_use. }
    OPTIONAL { wd:Q177 wdt:P2012 ?cuisine. }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
'''
)

In [ ]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

In [ ]:
# Example query for item Pizza
query = '''
SELECT ?origin ?culture ?ethnic_group ?off_language ?related_trad ?name ?country_use ?cuisine
WHERE {
    OPTIONAL { wd:Q177 wdt:P495 ?origin. }
    OPTIONAL { wd:Q177 wdt:P2596 ?culture. }
    OPTIONAL { wd:Q177 wdt:172 ?ethnic_group. }
    OPTIONAL { wd:Q177 wdt:37 ?off_language. }
    OPTIONAL { wd:Q177 wdt:4950 ?related_trad. }
    OPTIONAL { wd:Q177 wdt:407 ?name. }
    OPTIONAL { wd:Q177 wdt:17 ?country_use. }
    OPTIONAL { wd:Q177 wdt:P2012 ?cuisine. }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
'''

In [ ]:
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()


In [ ]:
print(data['results']['bindings'][0].keys())

In [ ]:
properties = ['origin', 'culture', 'ethnic_group', 'off_language',
               'related_trad', 'name', 'country_use', 'cuisine'
              ]
query_result = print_query_result(data, properties)

In [ ]:
table = pd.DataFrame.from_dict([query_result]) #  ,orient='index')
print(table)